## 全局参数设置

In [1]:
model_name_or_path = "/home/cc/models/asr/whisper-large-v2"  # 基础模型的存储路径
model_dir = "models/whisper-large-v2-asr-int8"  # 微调后模型的存储路径

language = "Chinese (China)"
language_abbr = "zh-CN"  # 加载的语言名称
task = "transcribe"  # 转录任务
dataset_name = "mozilla-foundation/common_voice_11_0"  # 数据集名称

batch_size=4  # 批次大小

save_path = "/home/cc/projects/Llm-quickstart/LLM-quickstart-main/mywork/week2/tokenized_common_voice"  # tokenized dataset的存储路径


本次微调使用的数据库和模型：  

mozilla-foundation/common_voice_11_0数据库：   
Common Voice 11.0 是 Mozilla 于 2022 年 9 月发布的第 11 版开源语音数据集。它是全球最大的多语言语音数据库之一，包含 24,210 小时的语音录音，覆盖 100 种语言（新增 4 种语言），其中 33 种语言的语音数据超过 100 小时。  
特点：  

    多语言支持：涵盖主流语言（如英语、中文、西班牙语）和小众语言（如阿塞拜疆语、科西嘉语），尤其注重低资源语言的覆盖。
    标注与质量：包含文本转录和发音评分，数据经过社区验证，适合训练高精度的语音识别模型。
    应用场景：常用于语音助手开发、多语言交互系统、无障碍技术（如实时字幕）等领域。


whisper-large-v2模型：
Whisper-large-v2 是 OpenAI 于 2022 年 12 月发布的自动语音识别（ASR）模型，基于 Transformer 架构。
特点：

    多语言支持：原生支持 90 余种语言，包括低资源语言（如斯瓦希里语、泰米尔语），并可通过微调进一步优化特定语言。
    多任务能力：
        语音识别：直接转录音频为同语言文本（如英语转英语）。
        语音翻译：将语音翻译成目标语言（如西班牙语转英语），零样本翻译准确率较高。
    长音频处理：通过分块算法支持任意长度音频的转录，适合处理讲座、播客等长内容。


### 加载数据集

把tokenized dataset处理完，存下来（参考save_dataset.ipynb），方便加载数据集和后续调试

In [2]:
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path) 
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, language=language, task=task)  
processor = AutoProcessor.from_pretrained(model_name_or_path, language=language, task=task)  

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



    AutoFeatureExtractor：用于处理非文本数据（如图像、音频）的特征提取器，能将原始非文本数据（如图像像素值、音频波形）转换为模型可处理的张量格式。
    AutoTokenizer：用于处理文本数据的分词器，能将原始文本拆分为模型可理解的 “令牌（token）”，并转换为对应的数字索引、注意力掩码（attention mask）等。
    AutoProcessor：是一个 “组合处理器”，通常用于需要处理多种类型数据的模型（如图文跨模态模型），它会整合AutoFeatureExtractor和AutoTokenizer的功能，提供统一的接口处理混合输入。


In [3]:
from datasets import load_from_disk

tokenized_common_voice = load_from_disk(save_path)  # 加载数据集

In [4]:
tokenized_common_voice

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'input_features', 'labels'],
        num_rows: 29056
    })
    validation: Dataset({
        features: ['audio', 'sentence', 'input_features', 'labels'],
        num_rows: 10581
    })
})

In [5]:
import torch


from dataclasses import dataclass  # 快速定义数据类（自动生成__init__等方法）
from typing import Any, Dict, List, Union

# 定义数据整理器类，用于处理语音seq2seq任务的数据填充逻辑
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    # 定义处理器属性，用于处理语音特征提取和文本token化（类型为Any，兼容各种processor）
    processor: Any  

    
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        # 使用processor的feature_extractor对"input_features"（语音的声学特征）进行填充（统一长度），并返回PyTorch张量格式的批量数据
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 对标签进行填充（统一长度）
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 将attention_mask中不为1的位置（即填充部分）的token id设为-100
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 检查所有标签的第一个token是否都是bos_token_id（句子开始符）
        # 如果是，则去除第一个token（部分模型生成时会自动添加bos_token，标签需对应调整）
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        # 将处理后的标签添加到批量数据中
        batch["labels"] = labels

        return batch


In [6]:
# 用给定的处理器实例化数据整理器
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

## 模型准备

### 加载预训练模型（int8 精度）

什么是 int8 模型？  
模型的参数（可以理解为模型学到的 “知识”）是用数字存储的。这些数字可以有不同的 “精度”：  

    比如 fp32（全精度）：数字更精确，但占的内存大、计算慢；  
    而 int8（低精度）：数字精度低一些，但占内存小、计算快，适合模型压缩和快速运行。  

In [7]:
from transformers import AutoModelForSpeechSeq2Seq

model = AutoModelForSpeechSeq2Seq.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")

### 模型生成配置调整  

在微调模型或使用模型生成文本时，我们常会调整模型的生成规则，让输出更符合需求。下面这两行代码就是在修改模型生成文本时的核心配置：  


##### 第一行：`model.config.forced_decoder_ids = None`  
先理解 `forced_decoder_ids` 是什么：  
它就像给模型的“强制任务清单”，里面记录着一些必须出现在生成结果中的词的编号（token ID）。比如在翻译任务中，可能会强制模型开头必须生成“翻译结果：”，这时候就需要在这个清单里写上对应词的编号。  

那设置为 `None` 是什么意思？  
就是告诉模型：“我没有任何强制要求，你不用必须包含某个特定的词，完全按自己的判断生成内容就行”。这样模型生成的文本会更自由，不会被固定的词语束缚。  


##### 第二行：`model.config.suppress_tokens = []`  
再看 `suppress_tokens` 是什么：  
它相当于模型的“禁止生成清单”，里面列着不允许出现在结果中的词的编号（比如脏话、敏感词）。模型生成时会自动避开这些词，确保输出更合规。  

那设置为 `[]`（空列表）是什么意思？  
就是告诉模型：“我没有任何要禁止的词，你生成任何内容都可以，不用刻意避开某个词”。这样模型不会有额外的生成限制，输出会更自然、完整。  


总结来说，这两行配置合起来的效果是：让模型在生成文本时“无强制要求、无禁止内容”，完全按照自身学到的知识自由生成，适合需要自然、不受约束输出的场景。

In [8]:
model.config.forced_decoder_ids = None 

model.config.suppress_tokens = [] 

### PEFT 微调前的模型处理
由于很多预训练好的大模型会被转成 int8 精度来节省空间，但如果要对它进行微调（用新数据让模型学新技能），直接用 int8 精度会有问题，需要先对模型做一些准备工作

三个预处理：

    1 把非 int8 的模块转成 fp32（全精度）
    一个模型可能不是所有部分都是 int8 精度（比如有些关键模块可能是其他精度）。训练时如果精度混乱，容易出现计算错误或不稳定（比如结果忽对忽错）。
    转成 fp32 就像给模型的 “关键零件” 换了更精密的齿轮，保证训练时计算稳定。
    2 给输入嵌入层加 forward_hook，开梯度计算
    模型处理数据时，第一步是把文字 / 图片等输入转换成数字（叫 “嵌入向量”），这个过程由 “嵌入层” 完成。
    微调时，模型需要根据训练结果调整参数（类似 “知错就改”），而调整的依据是 “梯度”（可以理解为 “错误的程度和方向”）。
    加 forward_hook 就像给嵌入层装了个 “传感器”，让模型能记录下输入转换过程中的梯度，这样后续才能正确调整参数。
    3 开梯度检查点，省内存
    训练大模型很费内存（因为要存很多中间计算结果）。
    梯度检查点的作用是：不保存所有中间结果，只在需要时重新计算，这样能大幅节省内存，让你的电脑/显卡能装下模型并完成训练（不然可能因为内存不够直接崩溃）。

prepare_model_for_int8_training 这个函数会帮你自动完成这些准备，不用手动操作。 

In [9]:
from peft import prepare_model_for_int8_training

model = prepare_model_for_int8_training(model)

/home/cc/.virtualenvs/peft/lib/python3.9/site-packages/peft/utils/other.py:141: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


### LoRA Adapter 配置
在使用 PEFT 工具进行参数高效微调时，LoRA（Low-Rank Adaptation，低秩适配）是一种常用方法——它不用微调模型所有参数，只需训练少量新增参数，就能让模型快速适应新任务，还能节省内存和计算资源。

### LoRA配置：高效微调的参数设定

LoraConfig：设定 LoRA 微调关键参数的配置工具，相当于给模型“定制一套高效学习的规则”。


### 核心参数详解

#### 1. r=4（LoRA的秩）
可以理解为“低维钥匙的齿纹复杂度”。

模型的原始参数矩阵通常很大（高维），LoRA 的思路是：用两个小的低秩矩阵（可以想象成两把小钥匙）去近似原始矩阵的更新量。这里的 `r` 就是这两个小矩阵的“秩”（维度大小）。

- `r` 越小：矩阵参数越少，计算越快、越省内存，但可能“学习能力有限”，复杂任务可能学不透；
- `r` 越大：矩阵能表达的信息越丰富，学习能力强，但参数变多，接近传统微调的消耗。
这里设为 4，属于轻量级配置，适合快速尝试或简单任务。


#### 2. lora_alpha=64（LoRA的比例因子）
可以理解为“学习强度的音量旋钮”。

LoRA 计算出的更新量（两个低秩矩阵的乘积）会先乘以 `lora_alpha`，再除以 `r`，最后才加到原始模型的参数上。这个参数控制着 LoRA 更新对模型的“影响力度”。

- `lora_alpha` 越大：相当于把 LoRA 学到的“新知识”放大后再融入模型，影响更显著；
- 通常会和 `r` 配合调整（比如这里 64/4=16，相当于整体放大 16 倍），平衡学习强度和稳定性。


#### 3. target_modules=["q_proj", "v_proj"]（目标模块）
可以理解为“给模型的哪些关键部件装上新的学习装置”。

大模型（比如 Transformer 架构）由很多模块组成，不是所有模块都需要微调。`target_modules` 用来指定：只在这些模块上应用 LoRA 机制。

这里选了 `q_proj`（查询投影）和 `v_proj`（值投影），它们是注意力机制的核心模块——模型通过“查询”找关键信息、通过“值”计算输出，微调这两个模块能高效提升模型对特定任务的“注意力聚焦能力”（比如理解特定领域的关键词）。


#### 4. lora_dropout=0.05（LoRA的dropout率）
可以理解为“学习时故意关掉部分小开关，防止死记硬背”。

`dropout` 是训练中常用的防过拟合技巧：随机让一部分神经元“暂时失效”（概率 5%），迫使模型不依赖特定参数，学会更通用的规律。

在 LoRA 模块中加 dropout，能让学到的“新知识”更稳健，避免模型只记住训练数据的细节，却不会举一反三。


#### 5. bias="none"（偏置项设置）
可以理解为“是否给学习装置加额外的小配重”。

`bias` 是模型中的偏置参数（类似公式里的常数项），用来微调输出的基准线。这里设为 `"none"`，表示不调整任何偏置项。

这么做的原因是：LoRA 本就是轻量微调，只关注核心参数的更新，省略偏置项可以进一步减少需要训练的参数，让过程更高效。


最后通过 `get_peft_model` 等函数，就能把这个配置应用到预训练模型上，实现高效微调。

In [10]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(
    r=4,  # LoRA的秩，影响LoRA矩阵的大小
    lora_alpha=64,  # LoRA适应的比例因子
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,  # 在LoRA模块中使用的dropout率
    bias="none",  # 设置bias的使用方式，这里没有使用bias
)

### 使用get_peft_model函数和给定的配置来获取一个PEFT模型

In [11]:
peft_model = get_peft_model(model, config)

In [12]:
# 打印 LoRA 微调训练的模型参数
peft_model.print_trainable_parameters()

trainable params: 1,966,080 || all params: 1,545,271,040 || trainable%: 0.12723204856023188


## 模型训练

#### Seq2SeqTrainingArguments 训练参数
    per_device_train_batch_size=batch_size,  # 在显存够用的情况下尽可能地大一点
    per_device_eval_batch_size=batch_size, 
    
    learning_rate=1e-3, # 学习率
    num_train_epochs=3,  # 训练轮次
    evaluation_strategy="epoch",
    generation_max_length=96,  # 生成的最大长度
    logging_steps=50,  #记日记的steps间隔
    remove_unused_columns=False,  # 是否删除某些列，可能会引起报错，最好为false
    label_names=["labels"],
    
    gradient_accumulation_steps=4,  # 累积4步再更新梯度，等效于batch_size=2×4=8
    
    # 混合精度参数微调（确保稳定性）
    fp16=True,
    fp16_opt_level="O1",  # O1比O2更稳定，适合显存紧张场景
    
    save_strategy="epoch",  # 按epoch保存模型，避免频繁保存
    load_best_model_at_end=True,  # 在最后加载最优模型

In [13]:
from transformers import Seq2SeqTrainingArguments

# 设置序列到序列模型训练的参数
training_args = Seq2SeqTrainingArguments(
    output_dir=model_dir,  # 指定模型输出和保存的目录
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size, 
    
    learning_rate=1e-3,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    generation_max_length=96,  
    logging_steps=50,
    remove_unused_columns=False,
    label_names=["labels"],
    gradient_accumulation_steps=4,  
    fp16=True,
    fp16_opt_level="O1", 
    save_strategy="epoch",
    load_best_model_at_end=True,
)

### 实例化 Seq2SeqTrainer 训练器

In [14]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=peft_model,
    train_dataset=tokenized_common_voice["train"],
    eval_dataset=tokenized_common_voice["validation"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
)
peft_model.config.use_cache = False

peft_model.config.use_cache = False 是一个针对训练阶段显存优化和梯度计算兼容性的关键设置，尤其在结合梯度检查点（gradient_checkpointing=True）时非常重要，具体作用如下：
1. use_cache 参数的含义  
    use_cache 控制模型在生成（generate）过程中是否缓存前一步的隐藏状态（hidden states）：  
    当 use_cache=True 时，模型会缓存每一步的隐藏状态，供下一步生成时直接复用，加速推理（生成）过程（适合推理阶段）。  
    当 use_cache=False 时，模型不缓存隐藏状态，每次生成都需要重新计算（适合训练阶段）。  

2. 训练时为何要设置 use_cache = False？  
    （1）与梯度检查点兼容
    如果之前的训练参数中启用了 gradient_checkpointing=True（显存优化的常用手段），则必须设置 use_cache=False：  
    梯度检查点的原理是不保存中间激活值，反向传播时重新计算，以节省显存；  
    而 use_cache=True 会强制保存隐藏状态，与梯度检查点的逻辑冲突，可能导致显存占用增加或计算错误。   
    （2）减少训练时的显存占用  
    训练阶段（尤其是 Seq2Seq 模型的训练）本身不需要缓存隐藏状态（生成逻辑在训练时通常通过 Teacher Forcing 实现，而非自回归生成），禁用缓存可以减少不必要的中间变量存储，进一步降低显存消耗。  
    （3）避免训练不稳定  
    部分模型（如 T5、Whisper）在训练时若开启 use_cache，可能导致梯度计算异常（因为缓存的状态未参与梯度流），进而影响模型收敛。  

In [15]:
trainer.train()

/home/cc/.virtualenvs/peft/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/cc/.virtualenvs/peft/lib/python3.9/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/home/cc/.virtualenvs/peft/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,0.047200,0.076384


/home/cc/.virtualenvs/peft/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/cc/.virtualenvs/peft/lib/python3.9/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/home/cc/.virtualenvs/peft/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/cc/.virtualenvs/peft/lib/python3.9/site-packa

TrainOutput(global_step=7264, training_loss=0.08966097092493909, metrics={'train_runtime': 9726.4992, 'train_samples_per_second': 2.987, 'train_steps_per_second': 0.747, 'total_flos': 6.1773119225856e+19, 'train_loss': 0.08966097092493909, 'epoch': 1.0})

### 保存 LoRA 模型(Adapter)

In [16]:
trainer.save_model(model_dir)

In [17]:
peft_model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 1280)
          (layers): ModuleList(
            (0-31): 32 x WhisperEncoderLayer(
              (self_attn): WhisperSdpaAttention(
                (k_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=False)
                (v_proj): lora.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1280, out_features=4, bias=False)
                  )
            

## 模型推理（可能需要重启 Notebook）

**再次加载模型会额外占用显存，如果显存已经达到上限，建议重启 Notebook 后再进行以下操作**


In [ ]:
model_dir = "models/whisper-large-v2-asr-int8"

language = "Chinese (China)"
language_abbr = "zh-CN"
language_decode = "chinese"
task = "transcribe"


### 使用 `PeftModel` 加载 LoRA 微调后 Whisper 模型

使用 `PeftConfig` 加载 LoRA Adapter 配置参数，使用 `PeftModel` 加载微调后 Whisper 模型

In [ ]:
from transformers import AutoModelForSpeechSeq2Seq, AutoTokenizer, AutoProcessor
from peft import PeftConfig, PeftModel

peft_config = PeftConfig.from_pretrained(model_dir)

base_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    peft_config.base_model_name_or_path,  # 从 PEFT 配置中获取基础模型路径
    load_in_8bit=True,  # 启用 8 位量化，减少显存占用（相比 32 位可节省 75% 显存）
    device_map="auto"  # 自动将模型分配到可用设备（优先 GPU，不足时用 CPU）
)


peft_model = PeftModel.from_pretrained(base_model, model_dir)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
processor = AutoProcessor.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
feature_extractor = processor.feature_extractor

### 使用 Pipeline API 部署微调后 Whisper 实现中文语音识别任务

In [20]:
test_audio = "test_zh.flac"
language_decode = "chinese"  # 与 Whisper 要求一致

In [21]:
from transformers import AutomaticSpeechRecognitionPipeline

pipeline = AutomaticSpeechRecognitionPipeline(model=peft_model, tokenizer=tokenizer, feature_extractor=feature_extractor)

forced_decoder_ids = processor.get_decoder_prompt_ids(language=language_decode, task=task)

In [22]:
import torch

with torch.cuda.amp.autocast():
    text = pipeline(test_audio, max_new_tokens=255)["text"]

/home/cc/.virtualenvs/peft/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [23]:
text

' 大家好，今天给大家带来一款重磅产品，性能提升了八十，但是价格只要有商的一半，这不仅仅是一个产品，更是一个精品，让我问问大家，这个价格你们觉得怎么样？'